In [ ]:
import nltk
import json

In [ ]:
import pandas as pd

flickr_df = {"example_id": [], "caption": [], "wav": [], "split": []}
flickr_base = "/data/user_data/sbharad2/SpeechCLIP/data/flickr/"
flickr_txt = f"{flickr_base}/Flickr8k.token.txt"
flickr_dev_mapping = f"{flickr_base}/Flickr_8k.devImages.txt"
flickr_test_mapping = f"{flickr_base}/Flickr_8k.testImages.txt"
flickr_train_mapping = f"{flickr_base}/Flickr_8k.trainImages.txt"


with open(flickr_txt, "r") as f:
    for line in f:
        example_id, caption = line.strip().split("	")
        example_id = example_id.strip()
        caption = caption.strip()
        flickr_df["example_id"].append(example_id)
        flickr_df["caption"].append(caption)
        flickr_df["wav"].append(
            example_id.replace("#", "_").replace(".jpg", "") + ".wav"
        )  # 1001773457_577c3a7d70.jpg#4 --> 1001773457_577c3a7d70_4.wav
        flickr_df["split"].append("train")

id2split = {}

with open(flickr_dev_mapping, "r") as f:
    for line in f:
        image_id = line.strip()
        id2split[image_id] = "dev"

with open(flickr_test_mapping, "r") as f:
    for line in f:
        image_id = line.strip()
        id2split[image_id] = "test"

with open(flickr_train_mapping, "r") as f:
    for line in f:
        image_id = line.strip()
        id2split[image_id] = "train"

c = 0
for i, example_id in enumerate(flickr_df["example_id"]):
    image_id, caption_id = example_id.split("#")
    try:
        flickr_df["split"][i] = id2split[image_id]
    except:
        c += 1

print(c)
flickr_df = pd.DataFrame(flickr_df)
flickr_df["image_id"] = flickr_df["example_id"].apply(lambda x: x.split("#")[0])
flickr_df.to_csv(f"{flickr_base}/flickr.csv", index=False)
flickr_df

460


,example_id,caption,wav,split,image_id
0,1000268201_693b08cb0e.jpg#0,A child in a pink dress is climbing up a set o...,1000268201_693b08cb0e_0.wav,train,1000268201_693b08cb0e.jpg
1,1000268201_693b08cb0e.jpg#1,A girl going into a wooden building .,1000268201_693b08cb0e_1.wav,train,1000268201_693b08cb0e.jpg
2,1000268201_693b08cb0e.jpg#2,A little girl climbing into a wooden playhouse .,1000268201_693b08cb0e_2.wav,train,1000268201_693b08cb0e.jpg
3,1000268201_693b08cb0e.jpg#3,A little girl climbing the stairs to her playh...,1000268201_693b08cb0e_3.wav,train,1000268201_693b08cb0e.jpg
4,1000268201_693b08cb0e.jpg#4,A little girl in a pink dress going into a woo...,1000268201_693b08cb0e_4.wav,train,1000268201_693b08cb0e.jpg
...,...,...,...,...,...
40455,997722733_0cb5439472.jpg#0,A man in a pink shirt climbs a rock face,997722733_0cb5439472_0.wav,test,997722733_0cb5439472.jpg
40456,997722733_0cb5439472.jpg#1,A man is rock climbing high in the air .,997722733_0cb5439472_1.wav,test,997722733_0cb5439472.jpg
40457,997722733_0cb5439472.jpg#2,A person in a red shirt climbing up a rock fac...,997722733_0cb5439472_2.wav,test,997722733_0cb5439472.jpg
40458,997722733_0cb5439472.jpg#3,A rock climber in a red shirt .,997722733_0cb5439472_3.wav,test,997722733_0cb5439472.jpg


In [20]:
flickr_df[flickr_df.split == "train"].shape, flickr_df[
    flickr_df.split == "dev"
].shape, flickr_df[flickr_df.split == "test"].shape

((30460, 4), (5000, 4), (5000, 4))

In [ ]:
sampled_df = {"example_id": [], "caption": [], "wav": [], "split": [], "image_id": []}
all_image_ids = flickr_df[flickr_df.split == "dev"].image_id.unique()
# sample 200 images
all_image_ids = all_image_ids[:200]

all_first_captions = []
for img_id in all_image_ids:
    img_df = flickr_df[flickr_df.image_id == img_id]
    sampled_df["example_id"].extend(img_df.example_id.tolist())
    sampled_df["caption"].extend(img_df.caption.tolist())
    sampled_df["wav"].extend(img_df.wav.tolist())
    sampled_df["split"].extend(img_df["split"].tolist())
    sampled_df["image_id"].extend(img_df["image_id"].tolist())
    all_first_captions.append(img_df.caption.tolist()[0])

sampled_df = pd.DataFrame(sampled_df)
print(len(all_first_captions))
sampled_df.to_csv(f"{flickr_base}/flickr_dev_sampled.csv", index=False)

200


In [32]:
def get_all_nouns(sentence):
    nouns = []
    for i, (word, postag) in enumerate(nltk.pos_tag(sentence.lower().split())):
        if postag == "NN":
            nouns.append(word)
    return nouns

In [ ]:
from collections import defaultdict, OrderedDict

all_dev_set_nouns = defaultdict(int)

for sentence in all_first_captions:
    for nn in get_all_nouns(sentence):
        all_dev_set_nouns[nn] += 1

all_dev_set_nouns = OrderedDict(
    sorted(all_dev_set_nouns.items(), key=lambda x: x[1], reverse=True)
)
all_dev_set_nouns

OrderedDict([('dog', 59),
             ('boy', 41),
             ('man', 31),
             ('girl', 25),
             ('water', 22),
             ('woman', 17),
             ('child', 11),
             ('air', 10),
             ('ball', 10),
             ('grass', 10),
             ('shirt', 9),
             ('front', 8),
             ('field', 6),
             ('group', 6),
             ('side', 6),
             ('pink', 6),
             ('jacket', 6),
             ('person', 5),
             ('baby', 5),
             ('tennis', 5),
             ('beach', 5),
             ('snow', 5),
             ('crowd', 5),
             ('orange', 5),
             ('blue', 5),
             ('rope', 5),
             ('jumping', 5),
             ('ground', 5),
             ('brown', 5),
             ('face', 5),
             ('mountain', 5),
             ('pool', 4),
             ('rock', 4),
             ('bench', 4),
             ('park', 4),
             ('mouth', 4),
             ('tree', 4),
  

In [42]:
# Sample 2k images from train such that each image has atleast one noun in the all_dev_set_nouns
train_sampled_df = {
    "example_id": [],
    "caption": [],
    "wav": [],
    "split": [],
    "image_id": [],
}
all_image_ids = flickr_df[flickr_df.split == "train"].image_id.unique()
print(len(all_image_ids))

for ii, img_id in enumerate(all_image_ids):
    img_df = flickr_df[flickr_df.image_id == img_id]
    include = False
    for i in range(5):
        if any(
            [
                nn in get_all_nouns(img_df.caption.tolist()[i])
                for nn in all_dev_set_nouns
            ]
        ):
            include = True
            include_idx = i
            break
    if include:
        train_sampled_df["example_id"].append(img_df.example_id.tolist()[include_idx])
        train_sampled_df["caption"].append(img_df.caption.tolist()[include_idx])
        train_sampled_df["wav"].append(img_df.wav.tolist()[include_idx])
        train_sampled_df["split"].append(img_df["split"].tolist()[include_idx])
        train_sampled_df["image_id"].append(img_df["image_id"].tolist()[include_idx])
    if len(train_sampled_df["image_id"]) >= 2000:
        break

train_sampled_df = pd.DataFrame(train_sampled_df)
train_sampled_df

6092
1 0
2 1
3 2
4 3
5 4
6 5
7 6
8 7
9 8
10 9
11 10
12 11
13 12
14 13
15 14
16 15
17 16
18 17
19 18
20 19
21 20
22 21
23 22
24 23
25 24
26 25
27 26
28 27
29 28
30 29
31 30
32 31
33 32
34 33
35 34
36 35
37 36
38 37
39 38
40 39
41 40
42 41
43 42
44 43
45 44
46 45
47 46
48 47
49 48
50 49
51 50
52 51
53 52
54 53
55 54
56 55
57 56
58 57
59 58
60 59
61 60
62 61
63 62
64 63
65 64
66 65
67 66
68 67
69 68
70 69
71 70
72 71
73 72
74 73
75 74
76 75
77 76
78 77
79 78
80 79
81 80
82 81
83 82
84 83
85 84
86 85
87 86
88 87
89 88
90 89
91 90
92 91
93 92
94 93
95 94
96 95
97 96
98 97
99 98
100 99
101 100
102 101
103 102
104 103
105 104
106 105
107 106
108 107
109 108
110 109
111 110
112 111
113 112
114 113
115 114
116 115
117 116
118 117
119 118
120 119
121 120
122 121
123 122
124 123
125 124
126 125
127 126
128 127
129 128
130 129
131 130
132 131
133 132
134 133
135 134
136 135
137 136
138 137
139 138
140 139
141 140
142 141
143 142
144 143
145 144
146 145
147 146
148 147
149 148
150 149
151 150
152 1

,example_id,caption,wav,split,image_id
0,1000268201_693b08cb0e.jpg#0,A child in a pink dress is climbing up a set o...,1000268201_693b08cb0e_0.wav,train,1000268201_693b08cb0e.jpg
1,1001773457_577c3a7d70.jpg#0,A black dog and a spotted dog are fighting,1001773457_577c3a7d70_0.wav,train,1001773457_577c3a7d70.jpg
2,1002674143_1b742ab4b8.jpg#0,A little girl covered in paint sits in front o...,1002674143_1b742ab4b8_0.wav,train,1002674143_1b742ab4b8.jpg
3,1003163366_44323f5815.jpg#0,A man lays on a bench while his dog sits by him .,1003163366_44323f5815_0.wav,train,1003163366_44323f5815.jpg
4,1007129816_e794419615.jpg#0,A man in an orange hat starring at something .,1007129816_e794419615_0.wav,train,1007129816_e794419615.jpg
...,...,...,...,...,...
1995,2695085862_2ed62df354.jpg#0,A bearded man with a Mohawk and skull bandanna .,2695085862_2ed62df354_0.wav,train,2695085862_2ed62df354.jpg
1996,2695961935_a2a6338f26.jpg#0,A black and white dog jumps over a hurdle .,2695961935_a2a6338f26_0.wav,train,2695961935_a2a6338f26.jpg
1997,2696060728_3043cfc38c.jpg#0,A girl stands on a surfboard which a man is st...,2696060728_3043cfc38c_0.wav,train,2696060728_3043cfc38c.jpg
1998,269630255_c3ec75c792.jpg#0,A brown dog is running across the grass with a...,269630255_c3ec75c792_0.wav,train,269630255_c3ec75c792.jpg


In [43]:
train_sampled_df["image_id"].nunique(), train_sampled_df["example_id"].nunique()

(2000, 2000)

In [52]:
train_sampled_df["target"] = train_sampled_df["caption"].apply(
    lambda x: [nn for nn in set(get_all_nouns(x)) if nn in all_dev_set_nouns]
)

train_sampled_df.to_csv(f"{flickr_base}/flickr_train_sampled.csv", index=False)

In [51]:
train_sampled_df.target.apply(lambda x: len(x)).value_counts()

target
2    743
1    500
3    485
4    187
5     65
6     14
7      6
Name: count, dtype: int64

In [57]:
len(all_dev_set_nouns)

287

In [ ]:
# Remove the nouns that are not present in the train_sampled_df, as they are not useful
c = 0
absent_dev_set_nouns = []
for tw in all_dev_set_nouns:
    present = False
    for tt in train_sampled_df.target:
        if tw in tt:
            present = True
            break
    if not present:
        c += 1
        absent_dev_set_nouns.append(tw)
c

62

In [ ]:
# Create targets for dev set
sampled_df["target"] = sampled_df["caption"].apply(
    lambda x: [
        nn
        for nn in set(get_all_nouns(x))
        if nn in all_dev_set_nouns and nn not in absent_dev_set_nouns
    ]
)

sampled_df.at[650, "target"] = [
    "dog"
]  # Manually correcting the target for this caption.
sampled_df

,example_id,caption,wav,split,image_id,target
0,1022454332_6af2c1449a.jpg#0,A child and a woman are at waters edge in a bi...,1022454332_6af2c1449a_0.wav,dev,1022454332_6af2c1449a.jpg,"[city, woman, child]"
1,1022454332_6af2c1449a.jpg#1,a large lake with a lone duck swimming in it w...,1022454332_6af2c1449a_1.wav,dev,1022454332_6af2c1449a.jpg,"[lone, lake, edge]"
2,1022454332_6af2c1449a.jpg#2,A little boy at a lake watching a duck .,1022454332_6af2c1449a_2.wav,dev,1022454332_6af2c1449a.jpg,"[boy, lake]"
3,1022454332_6af2c1449a.jpg#3,A young boy waves his hand at the duck in the ...,1022454332_6af2c1449a_3.wav,dev,1022454332_6af2c1449a.jpg,"[water, boy, hand, park]"
4,1022454332_6af2c1449a.jpg#4,"Two people are at the edge of a lake , facing ...",1022454332_6af2c1449a_4.wav,dev,1022454332_6af2c1449a.jpg,"[water, lake, city, edge]"
...,...,...,...,...,...,...
995,2394857899_76bfdf720b.jpg#0,A man in an orange construction uniform wears ...,2394857899_76bfdf720b_0.wav,dev,2394857899_76bfdf720b.jpg,"[hose, water, orange, construction, uniform, h..."
996,2394857899_76bfdf720b.jpg#1,A man in an orange safety suit spraying water .,2394857899_76bfdf720b_1.wav,dev,2394857899_76bfdf720b.jpg,"[water, orange, safety, suit, man]"
997,2394857899_76bfdf720b.jpg#2,A man in a public road workers uniform sprays ...,2394857899_76bfdf720b_2.wav,dev,2394857899_76bfdf720b.jpg,"[road, water, man]"
998,2394857899_76bfdf720b.jpg#3,A man wearing an orange uniform is working out...,2394857899_76bfdf720b_3.wav,dev,2394857899_76bfdf720b.jpg,"[hose, uniform, man, orange]"


In [ ]:
# Check if all test captions (caption number=0) have at least one target noun
first_cap = sampled_df.iloc[[_ for _ in range(0, 1000, 5)]]
first_cap.target.apply(len).value_counts()

target
3    59
2    59
4    38
1    17
5    16
6     6
7     5
Name: count, dtype: int64

In [107]:
# Store the sampled data
sampled_df.to_csv(f"{flickr_base}/flickr_dev_sampled.csv", index=False)

In [109]:
sampled_df.shape, train_sampled_df.shape

((1000, 6), (2000, 6))